# Equal-Weight S&P 500 Index Fund


**Disclaimer:** _Investing in the stock market involves risk and can lead to monetary loss. This material is purely for educational purposes and should not be taken as professional investment advice. Invest at your own discretion._

Based on the course [Algorithmic Trading Using Python - Full Course](https://www.youtube.com/watch?v=xfzGZB4HhEE) by Nick McCullum from Youtube channel [freeCodeCamp.org](https://www.youtube.com/channel/UC8butISFwT-Wl7EV0hUK0BQ).

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR S&P 500 ETF Trust. It has more than US$ 250 billion of assets under management.

The gola of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks
The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Payin for access to the index provider's API is outside the scope of this course.

There is a static version of the S&P 500 constituents available here. [Click the link to download them now](http://www.nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it is time to import these stocks into our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('./starter_files/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token
now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get published to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly generated and (more importnatly) has no cost associated with it.

[Click here]() to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
Now it's time to structure out API calls to IEX cloud.

We need the following information from the API
* Market capitalization per each stock (it is not actually used ?!)
* Price of each stock

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)

Check https://iexcloud.io/docs/api/ for details

In [5]:
print(data.status_code) # Should be 200

200


In [6]:
data.json()

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'RNGETMKBNCAEDLQE(SLAAL/OT S)AGS  ',
 'calculationPrice': 'iexlasttrade',
 'open': None,
 'openTime': None,
 'openSource': 'lciafoif',
 'close': None,
 'closeTime': None,
 'closeSource': 'folcaiif',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 141.329,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'February 2, 2021',
 'latestUpdate': 1653442444479,
 'latestVolume': None,
 'iexRealtimePrice': 134.98,
 'iexRealtimeSize': 104,
 'iexLastUpdated': 1617223108233,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 139.62,
 'previousVolume': 108702567,
 'change': 0.79,
 'changePercent': 0.00606,
 'volume': None,
 'iexMarketPercent': 0.016956484003549058,
 'iexVo

## Parsing Our API Call
The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy.

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
price = data.json()['latestPrice']
market_cap = data.json()['marketCap']
price, market_cap / 1000000000000

(141.329, 2.314023406075)

## Adding Our Stocks Data to a Pandas DataFrame
The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [9]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,141.329,2314023406075,N/A


## Looping Through The Tickers in Our List of Stocks
Using the same logic that we outlined above, we can pull data for all S&P stocks and store their data in the DataFrame using a `for` loop.

In [10]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe= final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index = my_columns
        ),
        ignore_index=True
    )

In [11]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.82,38341732792,N/A
1,AAL,17.20,10313212982,N/A
2,AAP,160.05,10789510399,N/A
3,AAPL,139.55,2332567207748,N/A
4,ABBV,104.85,184096859990,N/A
...,...,...,...,...
500,YUM,109.40,32738891950,N/A
501,ZBH,160.60,34023426091,N/A
502,ZBRA,413.51,22641664282,N/A
503,ZION,47.90,7763370500,N/A


In [12]:
final_dataframe.to_csv('sp500_stock_data.csv')

## Using Batch API Calls to Improve Performance
Batch API calls are one of the easiest ways to improve the performance of the code.

This is because HTTP requests are typically one of the slowest components of a scripts.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement in this section, we'll split our list of stocks of 100 and then make a batch API call for each group.

In [13]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i: i+n]

In [14]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [15]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe= final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )


In [16]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,126.560,38598523986,N/A
1,AAL,16.800,10287928923,N/A
2,AAP,157.040,10804078062,N/A
3,AAPL,139.185,2339660083551,N/A
4,ABBV,106.690,186803822063,N/A
...,...,...,...,...
500,YUM,107.800,32144170141,N/A
501,ZBH,164.600,34339640690,N/A
502,ZBRA,423.550,22464242340,N/A
503,ZION,47.240,7826876376,N/A


## Calculating the Number of Shares to Buy
As you can see in the DataFrame above, we still haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [17]:
while True:
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        val = float(portfolio_size)
        print(val)
        break
    except ValueError:
        print("That's not a number!\nPlease try again")

Enter the value of your portfolio:10000000
10000000.0


In [18]:
position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] \
        = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

In [19]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,126.560,38598523986,156
1,AAL,16.800,10287928923,1178
2,AAP,157.040,10804078062,126
3,AAPL,139.185,2339660083551,142
4,ABBV,106.690,186803822063,185
...,...,...,...,...
500,YUM,107.800,32144170141,183
501,ZBH,164.600,34339640690,120
502,ZBRA,423.550,22464242340,46
503,ZION,47.240,7826876376,419


## Formatting Our Excel Output
We will be using the XlsxWriter for Pyton to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [20]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats that We'll Need For Our `.xlsx` File
Formats include colors, fonts, and alos symbols like `%` and `$`. We'll need four main formats for our Excel document:

* String format for tickers
* `$XX.XX` format for stock prices
* `$XX.XXX` format for market capitalization
* Integer format for the numer of shares to purchase

In [21]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00', 
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File
We can use the `set_column` method applied to the `writer.book` object to apply formats to specific columns of our spreadsheets.

Here is an example:
```Python
   writer.book.set_column('B:B', # This tells the method to apply the format to column B
                          18, # This tells the method to apply a column wodth of 18 pixels
                          string_template # This applies the format `string_template` to the column
                         )
```

In [22]:
# Simple example
# The writer needs to be reinatialized to set new columns
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

# Set column titles
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

This code works, but it violates the software principle of "Don't repeat yourself"

Let's simplify this by putting it in two loops:

In [23]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalziation', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].write(f'{column}1',
                                              column_formats[column][0],
                                              column_formats[column][1])
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])


## Saving Our Excel Output
Saving our Excel file is very easy:

**THIS IS NOT AN INVESTMENT ADVICE!**

In [24]:
writer.save()